In [83]:
# TODO make
import os
import io
import re
import glob
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import json

In [74]:


def extract_tables_from_file(filepath, start_marker_FID, start_marker_TCD, end_marker):
    with open(filepath, 'r', encoding='utf-16', errors='ignore') as f:
        lines = f.readlines()

    FID_table_lines = []
    TCD_table_lines = []
    inside_table = False
    FID = False
    TCD = False

    for line in lines:
        if start_marker_FID in line:
            inside_table = True
            FID = True
            continue  # skip the start line itself
        elif start_marker_TCD in line:
            inside_table = True
            TCD = True
            continue
        elif 'Injection Date' in line:
            injection_time = datetime.strptime(re.search(r'Injection Date\s+: ([\d/]+\s+\d{1,2}:\d{2}:\d{2}\s+[AP]M)', line).group(1),"%m/%d/%Y %I:%M:%S %p")
            continue

        # TODO fix logic (it works just looks dumb)
        if inside_table:
            if line.strip().startswith(end_marker) and FID and not TCD:  # In FID table
                FID = False
                inside_table = False  # end of table
            elif line.strip().startswith(end_marker) and not FID and TCD: # In TCD table
                TCD = False
                inside_table = False  # end of table
                if FID:
                    print('FID is still inside_table')
                break
            elif FID and not TCD:
                FID_table_lines.append(line.strip())
            else:
                TCD_table_lines.append(line.strip())

    return (FID_table_lines, TCD_table_lines), injection_time


In [51]:
file_path = r"C:\Users\flahertylab\OneDrive - Georgia Institute of Technology\Flaherty Research Group - ESS\02 - PFR Data\ESS-1-151_Eyring_Plots\05kPa O2\0ppmCl_5kPaO2\001F0101.D\REPORT.txt"

tables, injection_time = extract_tables_from_file(
    file_path,
    start_marker_FID="Signal 1: FID1 A,",
    start_marker_TCD="Signal 2: TCD2 B,",
    end_marker="Totals :"
)

# Preview
print(injection_time)
print("\n".join(tables[0]))
#print("\n".join(TCD_table))


7/18/2025 9:26:06 AM

Peak RetTime Type  Width     Area      Height     Area
#   [min]        [min]   [pA*s]      [pA]         %
----|-------|----|-------|----------|----------|--------|
1   0.501 BB    0.0710    2.28594 4.55914e-1  0.01608
2   1.664 BB S  0.0266 8596.02734 5261.25635 60.45309
3   1.907 BB T  0.0288 5275.73340 2884.62646 37.10253
4   4.357 BB    0.0652    2.31922 5.66720e-1  0.01631
5   4.863 BB    0.0593    1.36984 3.60590e-1  0.00963
6   6.739 BV    0.0374  339.65970  144.85747  2.38872
7   6.882 VB    0.0394    1.93895 7.09660e-1  0.01364


In [69]:
def convert_to_dataframe(data_tables, injection_number=None, injection_time=None):
    # Skip header, units, and separator
    FID_data = data_tables[0][4:]
    TCD_data = data_tables[1][4:]
    #data_lines = table_lines[4:]

    FID_df = PFR_parse_data(FID_data)
    TCD_df = PFR_parse_data(TCD_data)

    for df in (FID_df, TCD_df):
        df['Injection #'] = injection_number
        df['Injection Time'] = injection_time

    return FID_df, TCD_df


def PFR_parse_data(data):
    parsed_data = []
    for line in data:

        line = line.strip()

        match = re.match(
            r"(\d+)\s+([\d.]+)\s+([A-Z ]+?)\s+([\d.]+)\s+([\d.eE+-]+)\s+([\d.eE+-]+)\s+([\d.]+)", 
            line
        )
        if match:
            parsed_data.append(match.groups())
        else:
            print(f"❌ Failed to parse: {repr(line)}")


    column_names = ['Peak', 'RetTime [min]', 'Type', 'Width', 'Area', 'Height', 'Percent']
    df = pd.DataFrame(parsed_data, columns=column_names)

    for col in ['Peak', 'RetTime [min]', 'Width', 'Area', 'Height', 'Percent']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df


In [33]:
dfs = convert_to_dataframe(tables)


1   0.501 BB    0.0710    2.28594 4.55914e-1  0.01608
2   1.664 BB S  0.0266 8596.02734 5261.25635 60.45309
3   1.907 BB T  0.0288 5275.73340 2884.62646 37.10253
4   4.357 BB    0.0652    2.31922 5.66720e-1  0.01631
5   4.863 BB    0.0593    1.36984 3.60590e-1  0.00963
6   6.739 BV    0.0374  339.65970  144.85747  2.38872
7   6.882 VB    0.0394    1.93895 7.09660e-1  0.01364
1   0.397 BB S  0.0356 9149.63770 3799.34985 69.00905
2   0.543 BB X  0.0357 1919.12671  793.68048 14.47458
3   1.499 BB    0.0691  311.79675   70.46693  2.35166
4   2.502 BB    0.1160  889.69891  120.65175  6.71035
5   3.285 BB    0.1584  590.78546   58.22143  4.45586
6   5.027 BB    0.1917  332.99402   24.79430  2.51153
7   8.240 BB    0.1128   64.56530    8.83389  0.48697


In [17]:
y = '01'
print(int(y))

1


In [ ]:
path = r"C:\Users\flahertylab\OneDrive - Georgia Institute of Technology\Flaherty Research Group - ESS\02 - PFR Data\ESS-1-151_Eyring_Plots\05kPa O2\0ppmCl_5kPaO2"

all_FID = []
all_TCD = []

for folder in os.listdir(path):
    if os.path.isdir(os.path.join(path,folder)) and folder.endswith('.D'):
        folder_path = os.path.join(path, folder)
        text_files = glob.glob(os.path.join(folder_path, 'REPORT.txt'))

        if not text_files:
            print(f'WARNING: {folder} does not have a REPORT file.')
            continue

        file_path  = text_files[0]

        with open(file_path, 'r', encoding='utf-16') as f:
            lines = f.readlines()
        
        data_tables, injection_time = extract_tables_from_file(file_path, start_marker_FID="Signal 1: FID1 A,", start_marker_TCD="Signal 2: TCD2 B,", end_marker="Totals :")
        
        
        injection_number, sequence_number = re.match(r'001F(\d{2})(\d{2})\.D', folder).groups()
        if int(sequence_number) != 1:
            print(f'WARNING: Sequence number {sequence_number} found, skipping folder')
            pass

        FID_df, TCD_df = convert_to_dataframe(data_tables, injection_number, injection_time)

        all_FID.append(FID_df)
        all_TCD.append(TCD_df)

FID_master = pd.concat(all_FID, ignore_index=True)
TCD_master = pd.concat(all_TCD, ignore_index=True)

TCD_master[TCD_master['Injection #']=='40'].head()

,Peak,RetTime [min],Type,Width,Area,Height,Percent,Injection #,Injection Time
278,1,0.390,BB S,0.0354,9121.90430,3807.711430,69.00903,40,2025-07-18 19:05:42
279,2,0.536,BB X,0.0384,1918.41248,788.308900,14.51317,40,2025-07-18 19:05:42
280,3,1.222,BB,0.0872,0.44707,0.065457,0.00338,40,2025-07-18 19:05:42
281,4,1.493,BB,0.0692,316.20712,71.322570,2.39217,40,2025-07-18 19:05:42
282,5,2.495,BB,0.1159,882.04437,119.796700,6.67284,40,2025-07-18 19:05:42


In [86]:
test_df = FID_master

In [ ]:
with open('../FID_peaks.json','r') as f:
    RetTime = json.load(f)

peak_dfs = {}

for peak_name, (low,high) in RetTime.items():
    peaks = test_df[low < test_df['RetTime [min]'] and high > test_df['RetTime [min]']]
    peak_dfs[peak_name] = peaks

print(peak_dfs['Ethylene'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().